In [92]:
#import dependencies
import pandas as pd
import numpy as np
import json
import re

In [9]:
file_dir="Data/"

In [10]:
with open(f'{file_dir}wikipedia-movies.json', mode='r') as file:
        wiki_movies_raw=json.load(file)

In [11]:
len(wiki_movies_raw)

7311

In [19]:
kaggle_metadata=pd.read_csv(f'{file_dir}movies_metadata.csv', low_memory=False)
ratings=pd.read_csv(f'{file_dir}ratings.csv')

In [24]:
kaggle_metadata[kaggle_metadata["original_language"]=='en'].sample(n=5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
30619,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,219666,tt2338874,en,Laiškai Sofijai,"Based on a true story, this is the love affair...",...,2013-08-28,0.0,0.0,"[{'iso_639_1': 'lt', 'name': 'Lietuvi\x9akai'}...",Released,A Lithuanian Love Affair,Letters To Sofija,False,6.0,2.0
35340,False,"{'id': 185667, 'name': 'The Complete Monterey ...",0,"[{'id': 10402, 'name': 'Music'}]",NaN,26028,tt0093312,en,Jimi Plays Monterey,It's no exaggeration to say this might be the ...,...,1986-10-23,0.0,50.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Jimi Plays Monterey,Jimi Plays Monterey,False,6.7,7.0
26972,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",NaN,79645,tt0067162,en,The Grissom Gang,The Grissom Gang is a remake of the notorious ...,...,1971-05-28,0.0,128.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"The psychotic killer, the young heiress...the ...",The Grissom Gang,False,6.9,7.0
15198,False,"{'id': 12077, 'name': ""St. Trinian's Collectio...",0,"[{'id': 35, 'name': 'Comedy'}, {'id': 12, 'nam...",NaN,37565,tt1210106,en,St Trinian's 2: The Legend of Fritton's Gold,The girls of St. Trinians are on the hunt for ...,...,2009-12-18,0.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,St Trinian's 2: The Legend of Fritton's Gold,False,5.4,73.0
6630,False,NaN,118,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",http://www.imdb.com/title/tt0078199/,37672,tt0078199,en,"Same Time, Next Year",A man and woman meet by chance at a romantic i...,...,1978-04-23,19.0,119.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,They couldn't have celebrated happier annivers...,"Same Time, Next Year",False,6.5,18.0


In [36]:
wiki_movies_df = pd.DataFrame(wiki_movies_raw)
len(wiki_movies)

0

In [67]:
wiki_movies = [movie for movie in wiki_movies_raw
              if ('Director' in movie or 'Directed by' in movie) 
                and 'imdb_link' in movie]

In [68]:
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    # combine alternate titles into one list
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune-Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles

    # merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')

    return movie

In [69]:
clean_movies = [clean_movie(movie) for movie in wiki_movies]
wiki_movies_df = pd.DataFrame(clean_movies)
sorted(wiki_movies_df.columns.tolist())

['Animation by',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Camera setup',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Created by',
 'Director',
 'Distributor',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'Genre',
 'Label',
 'Language',
 'McCune–Reischauer',
 'Narrated by',
 'No. of episodes',
 'Opening theme',
 'Original language(s)',
 'Original network',
 'Picture format',
 'Preceded by',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Recorded',
 'Release date',
 'Running time',
 'Starring',
 'Suggested by',
 'Venue',
 'Voices of',
 'Writer(s)',
 'alt_titles',
 'imdb_link',
 'title',
 'url',
 'year']

In [70]:
wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')

In [81]:
[[column,wiki_movies_df[column].isna().sum()] for column in wiki_movies_df]

[['url', 0],
 ['year', 0],
 ['imdb_link', 0],
 ['title', 1],
 ['Based on', 4889],
 ['Starring', 185],
 ['Narrated by', 6797],
 ['Cinematography', 700],
 ['Release date', 32],
 ['Running time', 140],
 ['Country', 239],
 ['Language', 248],
 ['Budget', 2315],
 ['Box office', 1563],
 ['Director', 0],
 ['Distributor', 362],
 ['Editor(s)', 551],
 ['Composer(s)', 520],
 ['Producer(s)', 203],
 ['Production company(s)', 1688],
 ['Writer(s)', 201],
 ['Genre', 6966],
 ['Original language(s)', 6918],
 ['Original network', 6951],
 ['Executive producer(s)', 6980],
 ['Production location(s)', 7030],
 ['Picture format', 7012],
 ['Audio format', 7016],
 ['Voices of', 7078],
 ['Followed by', 7071],
 ['Created by', 7069],
 ['Opening theme', 7079],
 ['No. of episodes', 7076],
 ['alt_titles', 7058],
 ['Preceded by', 7070],
 ['Suggested by', 7079],
 ['Recorded', 7078],
 ['Venue', 7079],
 ['Label', 7078],
 ['Animation by', 7078],
 ['Color process', 7079],
 ['Camera setup', 7079],
 ['McCune–Reischauer', 7078]

In [82]:
wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]

In [85]:
wiki_movies_df.dtypes

url                      object
year                      int64
imdb_link                object
title                    object
Based on                 object
Starring                 object
Cinematography           object
Release date             object
Running time             object
Country                  object
Language                 object
Budget                   object
Box office               object
Director                 object
Distributor              object
Editor(s)                object
Composer(s)              object
Producer(s)              object
Production company(s)    object
Writer(s)                object
imdb_id                  object
dtype: object

In [86]:
box_office = wiki_movies_df['Box office'].dropna()

In [91]:
box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)
box_office

0          $21.4 million
1           $2.7 million
2            $57,718,089
3             $7,331,647
4       $6,939,946 (USA)
              ...       
7074       $19.4 million
7075       $41.9 million
7076       $76.1 million
7077       $38.4 million
7078        $5.5 million
Name: Box office, Length: 5517, dtype: object

In [96]:
form_one = r'\$\d+\.?\d*\s*[mb]illion'
form_two = r'\$\d{1,3},(?:\d{3})+'

box_office.str.contains(form_one, flags=re.IGNORECASE, na=False).sum()

3913

In [97]:
box_office.str.contains(form_two, flags=re.IGNORECASE, na=False).sum()

1559